In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [45]:
tags = []
types = []

numerical_tag_list = list(range(1, 447))

#tags with modification: 68, 85, 86, 101, 125, 219 to 261

excluded_tags = list(range(219, 262)) + [68, 85, 86, 101, 125]
for i in excluded_tags:
    numerical_tag_list.remove(i)
numerical_tag_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 

In [46]:

for i in numerical_tag_list:
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/tagNum_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    tag_name = soup.find("h1").text.strip().split()[0]
    data_type = soup.find_all("p")[0].text.strip().split()[1]
    tags.append(tag_name)
    types.append(data_type)

In [47]:
d = {"tag": tags, "type":types, "numerical_tag": numerical_tag_list}
df = pd.DataFrame(d)
df

,tag,type,numerical_tag
0,Account,String,1
1,AdvId,String,2
2,AdvRefID,String,3
3,AdvSide,Char,4
4,AdvTransType,String,5
...,...,...,...
393,MultiLegReportingType,char,442
394,StrikeTime,UTCTimestamp,443
395,ListStatusText,String,444
396,EncodedListStatusTextLen,Length,445


In [48]:
df.value_counts(["type"])

type               
String                 112
int                     86
char                    46
Qty                     23
Price                   18
UTCTimestamp            16
Boolean                 15
data                    14
Length                  14
Amt                     13
float                   10
PriceOffset              6
LocalMktDate             5
Exchange                 5
Currency                 4
MultipleValueString      3
MonthYear                2
DayOfMonth               2
Quantity                 1
UTCDate                  1
UTCTimeOnly              1
Char                     1
dtype: int64

In [49]:
df[df['type']=="MonthYear"]

,tag,type,numerical_tag
194,MaturityMonthYear,MonthYear,200
264,UnderlyingMaturityMonthYear,MonthYear,313


In [50]:
#modify after size analysis

types_dict = {
    "String": "char*",
    "int":"uint64_t",
    "char": "char",
    "Qty": "uint64_t",
    "Price": "uint64_t",
    "UTCTimestamp": "uint64_t", #define a timestamp structure
    "Boolean": "bool",
    "data": "uint64_t",
    "Length": "uint64_t",
    "Amt": "uint64_t",
    "float": "double", #bytes?
    "PriceOffset": "uint64_t",
    "LocalMktDate": "uint64_t",
    "Exchange": "char*",
    "Currency": "char*",
    "MultipleValueString": "char*",
    "MonthYear": "uint64_t", #time data structure 
    "DayOfMonth": "uint64_t",
    "Quantity": "uint64_t",
    "UTCDate": "uint64_t",
    "UTCTimeOnly": "uint64_t",
    "Char": "char"
}

In [51]:
cpp_types = []
for i in types:
    cpp_types.append(types_dict[i])
# len(cpp_types), len(types)
df["cpp_type"] = cpp_types
df

,tag,type,numerical_tag,cpp_type
0,Account,String,1,char*
1,AdvId,String,2,char*
2,AdvRefID,String,3,char*
3,AdvSide,Char,4,char
4,AdvTransType,String,5,char*
...,...,...,...,...
393,MultiLegReportingType,char,442,char
394,StrikeTime,UTCTimestamp,443,uint64_t
395,ListStatusText,String,444,char*
396,EncodedListStatusTextLen,Length,445,uint64_t


In [52]:
#writing c++ struct
cpp_struct = "struct FIX_header{\n\t"
for i in range(len(tags)):
    cpp_struct += cpp_types[i] + " " + tags[i] + ";\n\t"
cpp_struct = cpp_struct[:-1] + "}"
print(cpp_struct)

struct FIX_header{
	char* Account;
	char* AdvId;
	char* AdvRefID;
	char AdvSide;
	char* AdvTransType;
	uint64_t AvgPx;
	uint64_t BeginSeqNo;
	char* BeginString;
	uint64_t BodyLength;
	char* CheckSum;
	char* ClOrdID;
	uint64_t Commission;
	char CommType;
	uint64_t CumQty;
	char* Currency;
	uint64_t EndSeqNo;
	char* ExecID;
	char* ExecInst;
	char* ExecRefID;
	char ExecTransType;
	char HandlInst;
	char* IDSource;
	char* IOIid;
	char IOIOthSvc;
	char IOIQltyInd;
	char* IOIRefID;
	char* IOIShares;
	char IOITransType;
	char LastCapacity;
	char* LastMkt;
	uint64_t LastPx;
	uint64_t LastShares;
	uint64_t LinesOfText;
	uint64_t MsgSeqNum;
	char* MsgType;
	uint64_t NewSeqNo;
	char* OrderID;
	uint64_t OrderQty;
	char OrdStatus;
	char OrdType;
	char* OrigClOrdID;
	uint64_t OrigTime;
	bool PossDupFlag;
	uint64_t Price;
	uint64_t RefSeqNum;
	char* RelatdSym;
	char Rule80A(aka;
	char* SecurityID;
	char* SenderCompID;
	char* SenderSubID;
	uint64_t SendingDate;
	uint64_t SendingTime;
	uint64_t Shares;


In [38]:
with open('cpp_struct.txt', 'w') as f:
    f.write(cpp_struct)

In [53]:
df[["tag", "numerical_tag"]]

,tag,numerical_tag
0,Account,1
1,AdvId,2
2,AdvRefID,3
3,AdvSide,4
4,AdvTransType,5
...,...,...
393,MultiLegReportingType,442
394,StrikeTime,443
395,ListStatusText,444
396,EncodedListStatusTextLen,445


In [54]:
df[["tag", "numerical_tag"]].to_csv("tags_numerical.csv", index=False)

Tags by message type (tag 35)

In [55]:
msg_types_1= []
tags_1 = []
fields = []
table_values_1 = []
msg_tags = {}

for i in range(10):
    print(f"message tag {i}")
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/msgType_{i}_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    msg = soup.find("h1").text.strip().split()[0]
    target_heading = soup.find("h3", text="Structure")
    target_table = target_heading.find_next_sibling("table")
    
    rows = target_table.find_all("tr")
    
    field_values = []
    print(f"row num: {len(rows)}")
    for r in range(2, len(rows)):
        cells = rows[r].find_all("td")
        # print(cells)
        # Process the cell data as needed
        cell_values = []
        for cell in cells:
            cell_values.append(cell.text.strip().split())
        if len(cell_values)>5:
            print(f"values length: {len(cell_values)}")
            print(cell_values)
        else: 
            field_values.append(cell_values)
    table_values_1.append(field_values)
    msg_types_1.append(msg)
    tags_1.append(i)
    
    

message tag 0
row num: 5
message tag 1
row num: 5
message tag 2
row num: 6
message tag 3
row num: 11
message tag 4
row num: 6
message tag 5
row num: 7
message tag 6
row num: 45
message tag 7
row num: 38
message tag 8
row num: 100
message tag 9
row num: 19


In [56]:
# list(range(97, 110))
ascii_list = list(range(97, 110)) + list(range(65, 91))
ascii_list.remove(73) #no 'I', 'U' or 'O' tag
ascii_list.remove(79)
ascii_list.remove(85)
print(ascii_list)

[97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90]


In [57]:
msg_types_2= []
tags_2 = []
table_values_2 = []


for i in ascii_list:
    # print(f"message tag {chr(i)}")
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/msgType_{chr(i)}_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    msg = soup.find("h1").text.strip().split()[0]
    target_heading = soup.find("h3", text="Structure")
    target_table = target_heading.find_next_sibling("table")
    
    rows = target_table.find_all("tr")
    
    field_values = []
    
    # print(f"row num: {len(rows)}")
    for r in range(2, len(rows)):
        cells = rows[r].find_all("td")
        # print(cells)
        # Process the cell data as needed
        cell_values = []
        for cell in cells:
            cell_values.append(cell.text.strip().split())
        if len(cell_values)>6:
            print(f"values length: {len(cell_values)}")
            print(cell_values)
        field_values.append(cell_values)
    table_values_2.append(field_values)
    msg_types_2.append(msg)
    tags_2.append(chr(i))

In [15]:
msg_types = msg_types_1 + msg_types_2
tag_values = tags_1 + tags_2
table_values = table_values_1 + table_values_2
len(msg_types_1), len(msg_types_2), len(table_values_1), len(table_values_2), len(tags_1), len(tags_2)

(10, 36, 10, 36, 10, 36)

In [58]:
len(table_values[0][0][0]), table_values[0][0][0][0]

(1, '<MessageHeader>')

In [59]:
table_values

[[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '0']],
  [['112'],
   ['TestReqID'],
   ['N'],
   ['Required',
    'when',
    'the',
    'heartbeat',
    'is',
    'the',
    'result',
    'of',
    'a',
    'Test',
    'Request',
    '<1>',
    'message.']],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '1']],
  [['112'], ['TestReqID'], ['Y'], []],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '2']],
  [['7'], ['BeginSeqNo'], ['Y'], []],
  [['16'], ['EndSeqNo'], ['Y'], []],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '3']],
  [['45'],
   ['RefSeqNum'],
   ['Y'],
   ['MsgSeqNum', '<34>', 'of', 'rejected', 'message']],
  [['371'],
   ['RefTagID'],
   ['N'],
   ['The',
    'tag',
    'number',
    'of',
    'the',
    'FIX',
    'field',
    'being',
    'referenced.']],
  [['372'],
   ['RefMsgType'],
   ['N'],
   ['The',
    'MsgType'

In [30]:
len(table_values[0][1])

4

In [60]:
fields = []
print(len(table_values))
for i in range(len(table_values)):
    print(table_values[i])
    msg_tags = []
    for j in range(len(table_values[i])):
        print(table_values[i][j])
        
        if len(table_values[i][j]) == 3:
            # msg_tags.append("header/trailer")
            print("len 3")
            # break
        elif len(table_values[i][j]) == 4:
            msg_tags.append(table_values[i][j][1][0])
            print(table_values[i][j][1][0])
        elif len(table_values[i][j]) == 5:
            msg_tags.append(table_values[i][j][2][0])
            print(table_values[i][j][2][0])
        elif len(table_values[i][j]) == 6:
            msg_tags.append(table_values[i][j][3][0])
            print(table_values[i][j][3][0])
        else:
            print(f"failed {len(table_values[i][j])}")
            print(table_values[i][j])
    fields.append(msg_tags)

46
[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '0']], [['112'], ['TestReqID'], ['N'], ['Required', 'when', 'the', 'heartbeat', 'is', 'the', 'result', 'of', 'a', 'Test', 'Request', '<1>', 'message.']], [['<MessageTrailer>'], ['Y'], []]]
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '0']]
len 3
[['112'], ['TestReqID'], ['N'], ['Required', 'when', 'the', 'heartbeat', 'is', 'the', 'result', 'of', 'a', 'Test', 'Request', '<1>', 'message.']]
TestReqID
[['<MessageTrailer>'], ['Y'], []]
len 3
[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '1']], [['112'], ['TestReqID'], ['Y'], []], [['<MessageTrailer>'], ['Y'], []]]
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '1']]
len 3
[['112'], ['TestReqID'], ['Y'], []]
TestReqID
[['<MessageTrailer>'], ['Y'], []]
len 3
[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '2']], [['7'], ['BeginSeqNo'], ['Y'], []], [['16'], ['EndSeqNo'], ['Y'], []], [['<MessageTrailer>'], ['Y'], []]]
[['<MessageHeader>'], ['Y'], ['MsgTyp

In [61]:
fields

[['TestReqID'],
 ['TestReqID'],
 ['BeginSeqNo', 'EndSeqNo'],
 ['RefSeqNum',
  'RefTagID',
  'RefMsgType',
  'SessionRejectReason',
  'Text',
  'EncodedTextLen',
  'EncodedText'],
 ['GapFillFlag', 'NewSeqNo'],
 ['Text', 'EncodedTextLen', 'EncodedText'],
 ['IOIid',
  'IOITransType',
  'IOIRefID',
  'Symbol',
  'SymbolSfx',
  'SecurityID',
  'IDSource',
  'SecurityType',
  'MaturityMonthYear',
  'MaturityDay',
  'PutOrCall',
  'StrikePrice',
  'OptAttribute',
  'ContractMultiplier',
  'CouponRate',
  'SecurityExchange',
  'Issuer',
  'EncodedIssuerLen',
  'EncodedIssuer',
  'SecurityDesc',
  'EncodedSecurityDescLen',
  'EncodedSecurityDesc',
  'Side',
  'IOIShares',
  'Price',
  'Currency',
  'ValidUntilTime',
  'IOIQltyInd',
  'IOINaturalFlag',
  'NoIOIQualifiers',
  'IOIQualifier',
  'Text',
  'EncodedTextLen',
  'EncodedText',
  'TransactTime',
  'URLLink',
  'NoRoutingIDs',
  'RoutingType',
  'RoutingID',
  'SpreadToBenchmark',
  'Benchmark'],
 ['AdvId',
  'AdvTransType',
  'AdvRefID'

In [67]:
struct_texts = []
failed = 0

for i in range(len(fields)):
    text = "struct FIX_message_" +msg_types[i]+"{\n\tstruct FIX_message_header;\n\t" 
    for j in range(len(fields[i])):
        # cpp_type = df[df["tag"]==fields[i][j]]["cpp_type"].iloc[0]
        try:
            tags_lower = [x.lower() for x in tags]
            cpp_type = cpp_types[tags_lower.index(fields[i][j].lower())]
            text += cpp_type + " " + fields[i][j] + ";\n\t"
        except:
            print(msg_types[i], fields[i][j])
            failed += 1
        
        
    text = text[:-1] + "}"
    struct_texts.append(text)
print(failed)

Indication ContractMultiplier
Indication CouponRate
Indication Benchmark
Advertisement ContractMultiplier
Advertisement CouponRate
Execution ContractMultiplier
Execution CouponRate
Quote ContractMultiplier
Quote CouponRate
Quote ContractMultiplier
Quote CouponRate
Security ContractMultiplier
Security CouponRate
Security ContractMultiplier
Security CouponRate
Security ContractMultiplier
Security CouponRate
Security ContractMultiplier
Security CouponRate
Mass ContractMultiplier
Mass CouponRate
List ContractMultiplier
List CouponRate
News ContractMultiplier
News CouponRate
Email ContractMultiplier
Email CouponRate
New ContractMultiplier
New CouponRate
New TotNoOrders(formerly
New ContractMultiplier
New CouponRate
Order ContractMultiplier
Order CouponRate
Order ContractMultiplier
Order CouponRate
Order ContractMultiplier
Order CouponRate
Allocation ContractMultiplier
Allocation CouponRate
List TotNoOrders(formerly
Don't ContractMultiplier
Don't CouponRate
Quote ContractMultiplier
Quote Cou

In [68]:
struct_texts

['struct FIX_message_Heartbeat{\n\tstruct FIX_message_header;\n\tchar* TestReqID;\n}',
 'struct FIX_message_Test{\n\tstruct FIX_message_header;\n\tchar* TestReqID;\n}',
 'struct FIX_message_Resend{\n\tstruct FIX_message_header;\n\tuint64_t BeginSeqNo;\n\tuint64_t EndSeqNo;\n}',
 'struct FIX_message_Reject{\n\tstruct FIX_message_header;\n\tuint64_t RefSeqNum;\n\tuint64_t RefTagID;\n\tchar* RefMsgType;\n\tuint64_t SessionRejectReason;\n\tchar* Text;\n\tuint64_t EncodedTextLen;\n\tuint64_t EncodedText;\n}',
 'struct FIX_message_Sequence{\n\tstruct FIX_message_header;\n\tbool GapFillFlag;\n\tuint64_t NewSeqNo;\n}',
 'struct FIX_message_Logout{\n\tstruct FIX_message_header;\n\tchar* Text;\n\tuint64_t EncodedTextLen;\n\tuint64_t EncodedText;\n}',
 'struct FIX_message_Indication{\n\tstruct FIX_message_header;\n\tchar* IOIid;\n\tchar IOITransType;\n\tchar* IOIRefID;\n\tchar* Symbol;\n\tchar* SymbolSfx;\n\tchar* SecurityID;\n\tchar* IDSource;\n\tchar* SecurityType;\n\tuint64_t MaturityMonthYear